# Comparison of activity estimators for a single source measured by a three detector setup

A radioactive source of unknown activity $x$ is put in front of 3 detectors with different instrinsic detection efficiencies (see figure below). The expected number for the detected counts $\bar{y}_1, \bar{y}_2, \bar{y}_3$ for a measurement with an acquisition time that is much shorter than the half life of the source is given by

$$\bar{y}_1(x) = a_1 \, x$$
$$\bar{y}_2(x) = a_2 \, x$$
$$\bar{y}_3(x) = a_3 \, x$$

where $a_1$, $a_2$, $a_3$ denote the total detection efficiencies (product of instrinsic efficiencies and acquisition time). In this notebook we assume $a_1>0$, $a_2>0$, and $a_3>0$.

![Figure 1: A source of unknown activity in front of three detectors](three_detectors.svg)

## Learning objectives of this notebook

1. **Implementing different estimators for the unknown activity of a source placed in front of three detectors yielding count measurements following a Poisson distribution.**
2. **Comparing different activity estimators in terms of expectation (bias) and variance (precision).**

## Important Instructions

### Task Overview

This notebook includes the following tasks, **make sure to complete all of them**:
- Task 1.1 and 1.2
- Task 2
- Task 3
- Task 4.1, 4.2.a, 4.2.b
- Task 5
- Task 6

### Programming Task Guidelines

- All programming tasks should be completed **within the provided code cells**
- **Do not create new cells**, as this will interfere with the autograding system (nbgrader).
- remove the `raise NotImplementedError` lines and insert your code there

**Note**: 
- Adhering to these instructions will ensure a smooth grading process.
- If anything is unclear, contact your instructor.


In [ ]:
# check wether the IPython version is new enough.
# We need IPython >= 3 for the autograding
import IPython
if IPython.version_info[0] < 3:
    raise ValueError("Your version of IPython is too old, please update it.")

In [ ]:
# import the module needed for this notebook
import numpy as np
import matplotlib.pyplot as plt

## Data simulation

We first simulate the outcome of many measurements for the hypothetical case of $x$ = 10.0Bq, $a_1$ = 3.0s, $a_2$ = 1.0s, $a_3$ = 0.25s.

**Note**: In real life, the **true value** of the **source activity is unknown**. Here we assume a hypothetical value to be able to simulate experiment outcomes that we can use to study different estimators.

In [ ]:
# ground truth (but in real life unknown) activity of the source, unit: [Bq]
x_true = 10.0

# define the total detection efficiencies of the three detectors
# product of acquisition time and intrinsic detector efficiencies
# unit: [s] - seconds

detector_sensitivities = np.array([3.0, 1.0, 0.25])
assert np.all(detector_sensitivities > 0), "Detectors sensitivities must be strictly positive."

# calculate the expected number of counts to be detected in every detector
expected_counts = x_true * detector_sensitivities 

for i in range(3):
    print(f"expectated number of counts to be measured by detector {i+1}: {expected_counts[i]}")

**Now we use a pseudo random generator to simulate the outcomes of 10 million experiments.**

In [ ]:
# generate many Poisson noise realizations of the measurement

# seed the random generator to ensure reproducible results
np.random.seed(1)

# the number of experiments to simulate
num_simulated_experiments = 10000000

# simulated number of counts seen in all three detectors for 1000000 repeated experiments
measured_counts1 = np.random.poisson(expected_counts[0], num_simulated_experiments)
measured_counts2 = np.random.poisson(expected_counts[1], num_simulated_experiments)
measured_counts3 = np.random.poisson(expected_counts[2], num_simulated_experiments)

# print the measured counts of the first 3 and last 3 experiments
for i in range(3):
    print(f"experiment {i:07}:   measured_counts1 = {measured_counts1[i]:02} counts,   measured_counts2 = {measured_counts2[i]:02} counts,   measured_counts3 = {measured_counts3[i]:02} counts")
print("...")
for i in range(num_simulated_experiments-2, num_simulated_experiments):
    print(f"experiment {i:07}:   measured_counts1 = {measured_counts1[i]:02} counts,   measured_counts2 = {measured_counts2[i]:02} counts,   measured_counts3 = {measured_counts3[i]:02} counts")

## (Naive) estimators to calculate the (unkown) activity from a single experiment

---

**To estimate the unknown activity of the source, one (naive) possibility is to use the only the measured counts by the detector 1 and to divide by the total sensitivity of detector 1**

$$\hat{x}_1 = \frac{y_1}{a_1}$$

**Another possibility is to use only the counts of measured by detector 2**

$$\hat{x}_2 = \frac{y_1}{a_2}$$

**or by detector 3**

$$\hat{x}_3 = \frac{y_1}{a_3}$$

---

**We implement the general form of the "single detector" activity estimator**

$$\hat{x}_i = \frac{y_i}{a_i} \quad i \in \{1,2,3\}$$

**in the cell below**

---

In [ ]:
def activity_estimator_single_detector(y_i: float | np.ndarray, a_i: float) -> float:
    """activity estimator using the only the measured counts from detector 1

    Parameters
    ---------

    y_i ... (float or numpy array)
        measured counts by detector i
    a_i ... (float)
        detection sensitivities if detector i
        product of acquisition time and intrinsic detection efficiency

    Returns
    -------

    float
    
    The estimated activity of the source
    """
    x_est = y_i/a_i

    return x_est

---

**Another (also naive) way of estimating the source activity, is to average the three naive estimator from above**

$$\hat{x}_\text{avg} = \frac{1}{3}(\hat{x}_1 + \hat{x}_2 + \hat{x}_3)$$

We implement this (naive) estimator in the following function

---

In [ ]:
def activity_estimator_single_detector_average(y_1: float, y_2: float, y_3: float, a_1: float, a_2: float, a_3: float) -> float:
    """estimator for the source activity using the average of the three individual estimators

    Parameters
    ---------

    y_1, y_2, y_3 ... (float)
        measured counts by detector 1, 2, and 3
    a_1, a_2, a_3 ... (float)
        detection sensitivities if detector 1, 2, and 3
        product of acquisition time and intrinsic detection efficiency

    Returns
    -------

    float
    
    The estimated activity of the source
    """
    x_est = (activity_estimator_single_detector(y_1, a_1) + 
             activity_estimator_single_detector(y_2, a_2) + 
             activity_estimator_single_detector(y_3, a_3)) / 3 

    return x_est

---

**Let's calculate the single detector estimator, as well as the single detector average estimator for all detectors for all our simulated experiments**

---

In [ ]:
x_est_det1_only = activity_estimator_single_detector(measured_counts1, detector_sensitivities[0])
x_est_det2_only = activity_estimator_single_detector(measured_counts2, detector_sensitivities[1])
x_est_det3_only = activity_estimator_single_detector(measured_counts3, detector_sensitivities[2])

x_est_single_det_avg = activity_estimator_single_detector_average(measured_counts1, measured_counts2, measured_counts3, detector_sensitivities[0], detector_sensitivities[1], detector_sensitivities[2])

---

**Let's print the results of the four naive estimators for the first 3 and last two simulated experiments**

---

In [ ]:
# print the numerical values of this estimator for the first 3 and last 2 simulated experiments
print(f"{' '*20} det1_only est 1 | det2_only est 2 | det3_only est 3 | avg est")
print("-"*82)
for i in range(3):
    print(f"experiment {i:07}:   {x_est_det1_only[i]:5.2f}{' '*11}| {x_est_det2_only[i]:5.2f}{' '*11}| {x_est_det3_only[i]:5.2f}{' '*11}| {x_est_single_det_avg[i]:5.2f}")
print("...")
for i in range(num_simulated_experiments-2, num_simulated_experiments):
    print(f"experiment {i:07}:   {x_est_det1_only[i]:5.2f}{' '*11}| {x_est_det2_only[i]:5.2f}{' '*11}| {x_est_det3_only[i]:5.2f}{' '*11}| {x_est_single_det_avg[i]:5.2f}")


---

**Based on the all simulated experiments, we can numerically calcuate the expectation and the variance of our 4 naive estimators**

---

In [ ]:
# calculate the mean our naive estimators
x_est_det1_only_numeric_expectation = x_est_det1_only.mean()
x_est_det2_only_numeric_expectation = x_est_det2_only.mean()
x_est_det3_only_numeric_expectation = x_est_det3_only.mean()
x_est_single_det_avg_numeric_expectation = x_est_single_det_avg.mean()

# calculate the sample variance (need ddof=1) our naive estimators
x_est_det1_only_numeric_var= x_est_det1_only.var(ddof=1)
x_est_det2_only_numeric_var= x_est_det2_only.var(ddof=1)
x_est_det3_only_numeric_var= x_est_det3_only.var(ddof=1)
x_est_single_det_avg_numeric_var= x_est_single_det_avg.var(ddof=1)

# display the numerically compuated values for the means and variances of the estimator
print(f"{' '*20} det1_only est 1 | det2_only est 2 | det3_only est 3 | avg est")
print("-"*82)
print(f"estimator exp.  .:   {x_est_det1_only_numeric_expectation:5.2f}{' '*11}| {x_est_det2_only_numeric_expectation:5.2f}{' '*11}| {x_est_det3_only_numeric_expectation:5.2f}{' '*11}| {x_est_single_det_avg_numeric_expectation:5.2f}")
print(f"estimator var.  .:   {x_est_det1_only_numeric_var:5.2f}{' '*11}| {x_est_det2_only_numeric_var:5.2f}{' '*11}| {x_est_det3_only_numeric_var:5.2f}{' '*11}| {x_est_single_det_avg_numeric_var:5.2f}")

---

**Let's visualize the distribution of the computed values of the 4 naive estimators across all experiments**

---

In [ ]:
fig, ax = plt.subplots(1,1, figsize = (9,4), tight_layout = True)
boxplot1 = ax.boxplot([x_est_det1_only,x_est_det2_only,x_est_det3_only,x_est_single_det_avg], 
                      labels = [f"single_det_1 ({x_est_det1_only_numeric_expectation:.1f}, {x_est_det1_only_numeric_var:.1f})", 
                                f"single_det_2 ({x_est_det2_only_numeric_expectation:.1f}, {x_est_det2_only_numeric_var:.1f})", 
                                f"single_det_3 ({x_est_det3_only_numeric_expectation:.1f}, {x_est_det3_only_numeric_var:.1f})", 
                                f"single_det_avg ({x_est_single_det_avg_numeric_expectation:.1f}, {x_est_single_det_avg_numeric_var:.1f})"],
                      showmeans = True,
                     flierprops = {"markersize":1})
ax.grid(ls = ":")
ax.set_xlabel("estimator type (expectation, variance)")
ax.set_ylabel("estimated source activity [Bq]")

---

**Observations:** 
1. We can see that **all** single detector **estimators** as well as the average single detector estimator have **the same mean value** (in other words, on average give the same results).
2. **However,** there are **strong differences in the variances** of these estimators.
3. The variance of the single detector estimator from detector 1 has lowest variance - even lower than the average single detector estimator.
   

---

## Analytic expressions for expectation and variance of the single detector estimator

Using basic arithmetic rules for the expectation and the variance, it is possible to show that the expectation of the single detector estimator is given by

$$
E[\hat{x}_i] = E[\frac{y_i}{a_i}] = \frac{1}{a_i}E[y_i] = \frac{1}{a_i} a_i x = x \ .
$$

The variance is given by

$$
Var[\hat{x}_i] = Var[\frac{y_i}{a_i}] = \frac{1}{a_i^2}Var[y_i] = \frac{1}{a_i^2}E[y_i] = \frac{x}{a_i} \ .
$$

We implement both analytic expressions in the cell below.

In [ ]:
def analytic_expectation_single_detector(x: float, a_i) -> float:
    """analytic value for the expectation of the single detector activity estimator $\\hat{x}_i$

    Parameters
    ---------

    x ... (float)
        the (unknown) true activity of the source
    a_i ... (float)
        detection sensitivities if detector i
        product of acquisition time and intrinsic detection efficiency

    Returns
    -------

    float
    
    The expectation of the estimator
    """

    return x

def analytic_variance_single_detector(x: float, a_i) -> float:
    """analytic value for the variance of the single detector activity estimator $\\hat{x}_i$

    Parameters
    ---------

    x ... (float)
        the (unknown) true activity of the source
    a_i ... (float)
        detection sensitivities if detector i
        product of acquisition time and intrinsic detection efficiency

    Returns
    -------

    float
    
    The expectation of the estimator
    """

    return x/a_i


---

**Let's calculate the analytic expressions of the single detector estimator expectation and variance.**

---

In [ ]:
for i in range(3):
    print(f"analytic expectation single detector estimator {i}: {analytic_expectation_single_detector(x_true, detector_sensitivities[i])}")

print()

for i in range(3):
    print(f"analytic variance    single detector estimator {i}: {analytic_variance_single_detector(x_true, detector_sensitivities[i])}")


**Observation**: We see that the **analytic values** for expectation and variance of the single detector estimator **closely match the numerical values** calculated above.

---
---
---

## Task 1.1

**Calculate and implement the analytic expressions for the expectation and variance of the single detector average estimator given by**

$$
\hat{x}_\text{avg} = \frac{1}{3}(\frac{y_1}{a_1} + \frac{y_2}{a_2} + \frac{y_3}{a_3})
$$

**in the cells below.**

---
---
---

In [ ]:
def analytic_expectation_single_detector_average(x: float, a_1: float, a_2: float, a_3: float) -> float:
    """analytic value for the expectation of the activity estimator $\\hat{x}_\\text{avg}$

    Parameters
    ---------

    x ... (float)
        the (unknown) true activity of the source
    a_1, a_2, a_3 ... (float)
        detection sensitivities if detector 1, 2, and 3
        product of acquisition time and intrinsic detection efficiency

    Returns
    -------

    float
    
    The expectation of the estimator
    """

    ########## YOUR CODE STARTS HERE #################################################
    # implement the expectation of the estimator as a general function (x, a_1, a_2, a_3)
    # return the correct expression as output of this function
    
    # YOUR CODE HERE
    raise NotImplementedError()
    ########## YOUR CODE ENDS HERE ###################################################

In [ ]:
# print the analytic value of the expectation for the parameter of our numerical experiments

print(f"analytic expectation of the single detector average estimator for x={x_true:.2f}, a_1 = {detector_sensitivities[0]:.2f}, a_2 = {detector_sensitivities[1]:.2f}, a_3 = {detector_sensitivities[2]:.2f}")
print(analytic_expectation_single_detector_average(x_true, detector_sensitivities[0], detector_sensitivities[1], detector_sensitivities[2]))

In [ ]:
# SKIP THIS BLANK CELL FOR AUTOGRADING

In [ ]:
def analytic_variance_single_detector_average(x: float, a_1: float, a_2: float, a_3: float) -> float:
    """analytic value for the variance of activity_estimator_avg

    Parameters
    ---------

    x ... (float)
        the (unknown) true activity of the source
    a_1, a_2, a_3 ... (float)
        detection sensitivities if detector 1, 2, and 3
        product of acquisition time and intrinsic detection efficiency

    Returns
    -------

    float
    
    The variance of the estimator
    """

    ########## YOUR CODE STARTS HERE #################################################
    # implement the variance of the estimator as a general function (x, a_1, a_2, a_3)
    # return the correct expression as output of this function
    
    # YOUR CODE HERE
    raise NotImplementedError()
    ########## YOUR CODE ENDS HERE ###################################################

In [ ]:
# print the analytic value of the variance for the parameter of our numerical experiments

print(f"analytic variance of single detector average estimator for x={x_true:.2f}, a_1 = {detector_sensitivities[0]:.2f}, a_2 = {detector_sensitivities[1]:.2f}, a_3 = {detector_sensitivities[2]:.2f}")
print(analytic_variance_single_detector_average(x_true, detector_sensitivities[0], detector_sensitivities[1], detector_sensitivities[2]))


In [ ]:
# SKIP THIS BLANK CELL FOR AUTOGRADING

## Task 1.2

---
---
---

What do you observe for the numerical and analytic expectation and variance of $\hat{x}_\text{avg}$?

**Question 1.2.a**: Is this estimator biased or unbiased (in other words, does the expectation match the true unknown value)?

**Question 1.2.b**: How does the variance of $\hat{x}_\text{avg}$ compare to the single detector estimator $\hat{x}_1$, $\hat{x}_2$, and $\hat{x}_3$? And why?

**Add your answers to questions 1 and 2 in the markdown cells below**

---
---
---

YOUR ANSWER HERE

YOUR ANSWER HERE

## Task 2

A (potentially) better estimator for the (unknown) activity of the source is the **Poisson Maximum likelihood estimator** given by value
$$
\DeclareMathOperator*{\argmax}{argmax}
\hat{x}_\text{ML,Poisson} = \argmax_x \sum_{i=1}^3 y_i \ln(\bar{y}_i(x)) - \bar{y}_i(x)
$$

**Implement the Poisson Maximum likelihood estimator for the activity of the source based on the counts of all 3 detectors by completing the function defined in the cell below.**

**The input to this function are the measured counts (random variables) and the sensitivities of all three detectors.**

In [ ]:
def activity_estimator_ml_poisson_function(y_1: float, y_2: float, y_3: float, a_1: float, a_2: float, a_3: float) -> float:
    """Poisson maximum likelihood estimator for the source activity using the counts measured by all detectors

    Parameters
    ---------

    y_1, y_2, y_3 ... (float)
        measured counts by detector 1, 2, and 3
    a_1, a_2, a_3 ... (float)
        detection sensitivities if detector 1, 2, and 3
        product of acquisition time and intrinsic detection efficiency

    Returns
    -------

    float
    
    The estimated activity of the source
    """
    
    # YOUR CODE HERE
    raise NotImplementedError()

In [ ]:
# SKIP THIS BLANK CELL FOR AUTOGRADING

In [ ]:
# apply the ML Poisson estimator to all simulated experiments
x_est_ml_poisson = activity_estimator_ml_poisson_function(measured_counts1, measured_counts2, measured_counts3, detector_sensitivities[0], detector_sensitivities[1], detector_sensitivities[2])
print(x_est_ml_poisson)

# numerically estimate the expectation and variance of the ML Poisson estimator
x_est_ml_poisson_numeric_expectation = x_est_ml_poisson.mean()
print(f"numeric value for expectation of ML Poisson estimator: {x_est_ml_poisson_numeric_expectation:.2f}")

x_est_ml_poisson_numeric_var = x_est_ml_poisson.var(ddof=1)
print(f"numeric value for var of ML Poisson estimator: {x_est_ml_poisson_numeric_var:.2f}")

## Task 3

Another (potentially) better estimator for the (unknown) activity of the source is the **least squares** given by value
$$
\DeclareMathOperator*{\argmax}{argmax}
\hat{x}_\text{least squares} = \argmax_x \sum_{i=1}^3 (y_i - \bar{y}_i(x))^2
$$

**Implement the least squares estimator for the activity of the source based on the counts of all 3 detectors by completing the function defined in the cell below.**

**The input to this function are the measured counts (random variables) and the sensitivities of all three detectors.**

In [ ]:
def activity_estimator_least_squares_function(y_1: float, y_2: float, y_3: float, a_1: float, a_2: float, a_3: float) -> float:
    """Poisson maximum likelihood estimator for the source activity using the counts measured by all detectors

    Parameters
    ---------

    y_1, y_2, y_3 ... (float)
        measured counts by detector 1, 2, and 3
    a_1, a_2, a_3 ... (float)
        detection sensitivities if detector 1, 2, and 3
        product of acquisition time and intrinsic detection efficiency

    Returns
    -------

    float
    
    The estimated activity of the source
    """
    
    # YOUR CODE HERE
    raise NotImplementedError()

In [ ]:
# SKIP THIS BLANK CELL FOR AUTOGRADING

In [ ]:
# apply the ML Poisson estimator to all simulated experiments
x_est_least_squares = activity_estimator_least_squares_function(measured_counts1, measured_counts2, measured_counts3, detector_sensitivities[0], detector_sensitivities[1], detector_sensitivities[2])
print(x_est_least_squares)

# numerically estimate the expectation and variance of the ML Poisson estimator
x_est_least_squares_numeric_expectation = x_est_least_squares.mean()
print(f"numeric value for expectation of least squares estimator: {x_est_least_squares_numeric_expectation:.2f}")

x_est_least_squares_numeric_var = x_est_least_squares.var(ddof=1)
print(f"numeric value for variance of least squares estimator: {x_est_least_squares_numeric_var:.2f}")

---
---
---

## Task 4.1

**Visualize** the calculated values of the **Poisson Maximum Likelihood and least squares estimate** next to the single detector estimators by completing the **box plot below**.

---
---
---

In [ ]:
fig, ax = plt.subplots(1,1, figsize = (12,4), tight_layout = True)

#boxplot2 = ax.boxplot([x_est_det1_only,x_est_det2_only,x_est_det3_only,x_est_single_det_avg], 
#                      labels = [f"single_det_1 ({x_est_det1_only_numeric_expectation:.1f}, {x_est_det1_only_numeric_var:.1f})", 
#                                f"single_det_2 ({x_est_det2_only_numeric_expectation:.1f}, {x_est_det2_only_numeric_var:.1f})", 
#                                f"single_det_3 ({x_est_det3_only_numeric_expectation:.1f}, {x_est_det3_only_numeric_var:.1f})", 
#                                f"single_det_avg ({x_est_single_det_avg_numeric_expectation:.1f}, {x_est_single_det_avg_numeric_var:.1f})"],
#                      showmeans = True,
#                     flierprops = {"markersize":1})


# YOUR CODE HERE
raise NotImplementedError()
ax.grid(ls = ":")
ax.set_ylabel("estimated source activity [Bq]")
ax.set_xlabel("estimator type (expectation, variance)")


---
---
---

## Task 4.2

4.2.a **Observation Analysis**  
Describe the numerical values of the **expectation** and **variance** for the Poisson Maximum Likelihood and the Least Squares estimators based on the simulated experiments.

4.2.b **Comparison**  
Compare these values against the estimators obtained using data from individual detectors. Discuss which estimator appears to be most precise. Why is that the case?

---
---
---

YOUR ANSWER HERE

YOUR ANSWER HERE

## Task 5

For the **Poisson maximum likelihood and the least squares estimators** above, it is also possible to derive an **analytic expression for their expectation and their variance**.

Implement these analytic expressions into the functions defined in the 4 cells below. 


In [ ]:
def analytic_expectation_ml_poisson(x: float, a_1: float, a_2: float, a_3: float) -> float:
    """analytic value for the expectation of activity_estimator_ml_poisson

    Parameters
    ---------

    x ... (float)
        the (unknown) true activity of the source
    a_1, a_2, a_3 ... (float)
        detection sensitivities if detector 1, 2, and 3
        product of acquisition time and intrinsic detection efficiency

    Returns
    -------

    float
    
    The expectation of the estimator
    """

    # YOUR CODE HERE
    raise NotImplementedError()

In [ ]:
# SKIP THIS BLANK CELL FOR AUTOGRADING

In [ ]:
# print the analytic value of the expectation for the parameter of our numerical experiments
print(f"analytic expectation of the ML Poisson estimator for x={x_true:.2f}, a_1 = {detector_sensitivities[0]:.2f}, a_2 = {detector_sensitivities[1]:.2f}, a_3 = {detector_sensitivities[2]:.2f}")
print(analytic_expectation_ml_poisson(x_true, detector_sensitivities[0], detector_sensitivities[1], detector_sensitivities[2]))

In [ ]:
def analytic_variance_ml_poisson(x: float, a_1: float, a_2: float, a_3: float) -> float:
    """analytic value for the variance of activity_estimator_ml_poisson

    Parameters
    ---------

    x ... (float)
        the (unknown) true activity of the source
    a_1, a_2, a_3 ... (float)
        detection sensitivities if detector 1, 2, and 3
        product of acquisition time and intrinsic detection efficiency

    Returns
    -------

    float
    
    The variance of the estimator
    """

    # YOUR CODE HERE
    raise NotImplementedError()

In [ ]:
# SKIP THIS BLANK CELL FOR AUTOGRADING

In [ ]:
# print the analytic value of the variance for the parameter of our numerical experiments
print(f"analytic variance of the ML Poisson estimator for x={x_true:.2f}, a_1 = {detector_sensitivities[0]:.2f}, a_2 = {detector_sensitivities[1]:.2f}, a_3 = {detector_sensitivities[2]:.2f}")
print(analytic_variance_ml_poisson(x_true, detector_sensitivities[0], detector_sensitivities[1], detector_sensitivities[2]))

In [ ]:
def analytic_expectation_least_squares(x: float, a_1: float, a_2: float, a_3: float) -> float:
    """analytic value for the expectation of activity_estimator_least_squares

    Parameters
    ---------

    x ... (float)
        the (unknown) true activity of the source
    a_1, a_2, a_3 ... (float)
        detection sensitivities if detector 1, 2, and 3
        product of acquisition time and intrinsic detection efficiency

    Returns
    -------

    float
    
    The expectation of the estimator
    """

    # YOUR CODE HERE
    raise NotImplementedError()

In [ ]:
# SKIP THIS BLANK CELL FOR AUTOGRADING

In [ ]:
# print the analytic value of the expectation for the parameter of our numerical experiments
print(f"analytic expectation of the least squares estimator for x={x_true:.2f}, a_1 = {detector_sensitivities[0]:.2f}, a_2 = {detector_sensitivities[1]:.2f}, a_3 = {detector_sensitivities[2]:.2f}")
print(analytic_expectation_least_squares(x_true, detector_sensitivities[0], detector_sensitivities[1], detector_sensitivities[2]))

In [ ]:
def analytic_variance_least_squares(x: float, a_1: float, a_2: float, a_3: float) -> float:
    """analytic value for the variance of activity_estimator_least_squares

    Parameters
    ---------

    x ... (float)
        the (unknown) true activity of the source
    a_1, a_2, a_3 ... (float)
        detection sensitivities if detector 1, 2, and 3
        product of acquisition time and intrinsic detection efficiency

    Returns
    -------

    float
    
    The variance of the estimator
    """

    # YOUR CODE HERE
    raise NotImplementedError()

In [ ]:
# SKIP THIS BLANK CELL FOR AUTOGRADING

In [ ]:
# print the analytic value of the variance for the parameter of our numerical experiments
print(f"analytic variance of the least squares estimator for x={x_true:.2f}, a_1 = {detector_sensitivities[0]:.2f}, a_2 = {detector_sensitivities[1]:.2f}, a_3 = {detector_sensitivities[2]:.2f}")
print(analytic_variance_least_squares(x_true, detector_sensitivities[0], detector_sensitivities[1], detector_sensitivities[2]))

---
---
---

## Task 6

Summarize in your own words, what you learned by completing this notebook. Use the markdown cell below.

---
---
---

YOUR ANSWER HERE